In [ ]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from config import RIVAL10_CONFIG as config_dict
from config import PROJECT_ROOT
import numpy as np

# Load and Transform Data

In [2]:
# # CUB DATASET

# # C_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'C_train_instance.npy'))
# # C_hat_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'C_hat_sigmoid_train_instance.npy'))
# # one_hot_Y_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'Y_train_instance.npy'))

# # C_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'C_test_instance.npy'))
# # C_hat_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'C_hat_sigmoid_test_instance.npy'))
# # one_hot_Y_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'Y_test_instance.npy'))

# C_hat_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'C_hat_sigmoid_train.npy'))
# one_hot_Y_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'Y_train.npy'))

# C_hat_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'C_hat_sigmoid_test.npy'))
# one_hot_Y_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'Y_test.npy'))

# class_level_concepts = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'class_level_concepts.npy'))

In [3]:
# # Derm7pt DATASET
# DERM7PT_PATH = os.path.join(PROJECT_ROOT, 'output', 'Derm7pt')

# C_hat_train = np.load(os.path.join(DERM7PT_PATH, 'C_hat_sigmoid_train.npy'))
# one_hot_Y_train = np.load(os.path.join(DERM7PT_PATH, 'Y_train.npy'))

# C_hat_test = np.load(os.path.join(DERM7PT_PATH, 'C_hat_sigmoid_test.npy'))
# one_hot_Y_test = np.load(os.path.join(DERM7PT_PATH, 'Y_test.npy'))

# class_level_concepts = np.load(os.path.join(DERM7PT_PATH, 'class_level_concepts.npy'))

In [4]:
# RIVAL10 DATASET
RIVAL10_PATH = os.path.join(PROJECT_ROOT, 'output', 'RIVAL10')

C_hat_train = np.load(os.path.join(RIVAL10_PATH, 'C_hat_sigmoid_train.npy'))
one_hot_Y_train = np.load(os.path.join(RIVAL10_PATH, 'Y_train.npy'))

C_hat_test = np.load(os.path.join(RIVAL10_PATH, 'C_hat_sigmoid_test.npy'))
one_hot_Y_test = np.load(os.path.join(RIVAL10_PATH, 'Y_test.npy'))

class_level_concepts = np.load(os.path.join(RIVAL10_PATH, 'class_level_concepts.npy'))

In [5]:
Y_train = np.argmax(one_hot_Y_train, axis=1)
Y_test = np.argmax(one_hot_Y_test, axis=1)

In [6]:
C_train = []
for y in Y_train:
    C_train.append(class_level_concepts[y])

C_train = np.array(C_train)

In [7]:
from sklearn.utils import shuffle


C_hat_train, C_train, one_hot_Y_train, Y_train = shuffle(C_hat_train, C_train, one_hot_Y_train, Y_train, random_state=42)

In [8]:
# C_hat_train.shape, one_hot_Y_train.shape, C_hat_test.shape, one_hot_Y_test.shape

In [9]:
# C_hat_train[C_hat_train < 0.1] = 0
# C_hat_test[C_hat_test < 0.1] = 0

# Classic Models

## Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(C_hat_train, Y_train)
print(f"Logistic Regression Test accuracy: {model.score(C_hat_test, Y_test)}")

Logistic Regression Test accuracy: 0.9568671963677639


## k-NN

In [11]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(C_hat_train, Y_train)
print(f"k-NN Test accuracy: {model.score(C_hat_test, Y_test)}")

k-NN Test accuracy: 0.9559213015512675


## Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(C_hat_train, Y_train)
print(f"Decision Tree Test accuracy: {model.score(C_hat_test, Y_test)}")

Decision Tree Test accuracy: 0.9462731744230042


## MLP

In [13]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(512,256, 128), max_iter=1000)
mlp.fit(C_hat_train, Y_train)
print(f"MLP Test accuracy: {mlp.score(C_hat_test, Y_test)}")

MLP Test accuracy: 0.9547862277714718


# Accuracy Using Class-Level Concepts

In [14]:
# Function to find the closest concept vector and predict the label
def predict_nearest_concept(instance, reference_concepts, reference_labels):
    distances = np.sqrt(np.sum((reference_concepts - instance)**2, axis=1))
    min_idx = np.argmin(distances)
    return reference_labels[min_idx]

# Use C_train as reference concepts and evaluate on C_hat_test
correct_predictions = 0
total_predictions = len(C_hat_test)

for i, test_instance in enumerate(C_hat_test):
    predicted_label = predict_nearest_concept(test_instance, C_train, Y_train)
    true_label = Y_test[i]

    if predicted_label == true_label:
        correct_predictions += 1

# Calculate and print accuracy
accuracy = correct_predictions / total_predictions
print(f"\nOverall accuracy using concept-based nearest neighbor: {accuracy:.4f}")


Overall accuracy using concept-based nearest neighbor: 0.9565


# Prototype-Based Model


In [15]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


## Create Dataloaders

In [17]:
val_split_ratio = 0.2
random_seed = 42

C_hat_train, C_hat_val, Y_train, Y_val = train_test_split(C_hat_train, one_hot_Y_train, test_size=val_split_ratio, random_state=random_seed)

X_train = torch.tensor(C_hat_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)

X_val = torch.tensor(C_hat_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)

X_test = torch.tensor(C_hat_test, dtype=torch.float32, device=device)
Y_test = torch.tensor(one_hot_Y_test, dtype=torch.float32, device=device)

batch_size = 64
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Learn Prototypes

In [ ]:
from src.models import PrototypeClassifier

num_concepts = config_dict['N_TRIMMED_CONCEPTS']
num_classes = config_dict['N_CLASSES']

model = PrototypeClassifier(num_concepts, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lambda_binary = 0.01
lambda_L1 = 0.001

In [19]:
# train and test
from src.training import train_epoch, val_epoch

num_epochs = 200

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_loader, optimizer, lambda_binary, lambda_L1, device=device)
    val_accuracy = val_epoch(model, val_loader, device)
    if((epoch+1)%10==0):
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

Epoch [10/200]
Train Loss: 0.4366, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [20/200]
Train Loss: 0.4120, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [30/200]
Train Loss: 0.4101, Train Accuracy: 95.72%, Validation Accuracy: 96.07%
Epoch [40/200]
Train Loss: 0.4099, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [50/200]
Train Loss: 0.4096, Train Accuracy: 95.72%, Validation Accuracy: 96.07%
Epoch [60/200]
Train Loss: 0.4100, Train Accuracy: 95.72%, Validation Accuracy: 96.07%
Epoch [70/200]
Train Loss: 0.4100, Train Accuracy: 95.70%, Validation Accuracy: 96.07%
Epoch [80/200]
Train Loss: 0.4096, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [90/200]
Train Loss: 0.4097, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [100/200]
Train Loss: 0.4097, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [110/200]
Train Loss: 0.4098, Train Accuracy: 95.71%, Validation Accuracy: 96.07%
Epoch [120/200]
Train Loss: 0.4101, Train

In [20]:
real_labels = Y_test.argmax(dim=1)
predictions = model.predict(X_test)
(predictions == real_labels).sum().item()/len(predictions)

0.9564888384411654

# MY OLD CODE

In [21]:
close_to_zero = (torch.sum((model.get_sigmoid_prototypes() < 0.1) | (model.get_sigmoid_prototypes() > 0.9)) / (200*112)).cpu().numpy()
print(f"{close_to_zero*100}% of the values are close to 0 or 1")

0.8035714626312256% of the values are close to 0 or 1


In [22]:
# # --- Plotting ---
# from matplotlib import pyplot as plt

# plt.figure(figsize=(10, 5))
# epochs_range = range(1, epochs + 1)
# plt.plot(epochs_range, train_losses, label='Training Loss', marker='o', linestyle='-')
# plt.plot(epochs_range, val_losses, label='Validation Loss', marker='x', linestyle='--')
# plt.title('Training and Validation Loss Over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Average Loss')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Optional: Plot validation accuracy as well
# plt.figure(figsize=(10, 5))
# plt.plot(epochs_range, val_accuracies, label='Validation Accuracy', marker='s', linestyle='-', color='green')
# plt.title('Validation Accuracy Over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.grid(True)
# plt.show()

In [23]:
# prototypes = []
# for y in Y_train:
#     prototypes.append(final_binary_prototypes[y])

# prototypes = np.array(prototypes)

In [24]:
# # Function to find the closest concept vector and predict the label
# def predict_nearest_concept(instance, reference_concepts, reference_labels):
#     distances = np.sum(np.abs(reference_concepts - instance), axis=1)
#     min_idx = np.argmin(distances)
#     return reference_labels[min_idx]

# # Use prototypes as reference concepts and evaluate on C_hat_test
# correct_predictions = 0
# total_predictions = len(C_hat_test)

# for i, test_instance in enumerate(C_hat_test):
#     predicted_label = predict_nearest_concept(test_instance, prototypes, Y_train)
#     true_label = Y_test[i]

#     if predicted_label == true_label:
#         correct_predictions += 1

# # Calculate and print accuracy
# accuracy = correct_predictions / total_predictions
# print(f"\nOverall accuracy using prototype-based nearest neighbor: {accuracy:.4f}")